# Retrieve Data

This notebook explores and downloads a dataset that meets the following criteria:
* `YEAR` = 2024
* `MONTH` is between March and July

In [41]:
from sqlalchemy import create_engine
from pathlib import Path
import pandas as pd
import helpers as h
from pathlib import Path
import os
import importlib
import helpers as h
import duckdb
import geopandas as gpd
from shapely import wkt

Using `pgadmin`, I found that there are 21,946,235 rows that match the date range criteria.

Download the first 100,000 rows of the dataset slice:

In [ ]:
@h.Timer()
def retrieve_data():
    """Downloads appropriate selection of data from PostgreSQL database"""

    with open('data/user.txt', 'r') as file:
        user = file.read().strip('\n')

    with open('data/pass.txt', 'r') as file:
        pw = file.read().strip('\n')

    with open('data/db_host.txt', 'r') as file:
        host = file.read().strip('\n')

    with open('data/db_port.txt', 'r') as file:
        port = file.read().strip('\n')

    with open('data/db_name.txt', 'r') as file:
        name = file.read().strip('\n')

    engine = create_engine(f'postgresql://{user}:{pw}@{host}:{port}/{name}')

    query = """SELECT * FROM blob WHERE "YEAR" = '2024' AND "MONTH" BETWEEN '03' AND '07' limit 100000;"""

    df = pd.read_sql(query, engine)

    parquet_path = Path('data/blob_data_100k.parquet')
    df.to_parquet(parquet_path, engine='pyarrow')

    print(f'Data saved to: {parquet_path}')

In [ ]:
# run function
retrieve_data()

Data saved to: data/blob_data_100k.parquet
Function `retrieve_data` executed in 654.5031 sec, CPU: 2.50%, Memory: 1362.84MB


In [ ]:
# confirm count of created dataset

parquet_path = Path('data/blob_data_100k.parquet')

check = duckdb.sql(
        f"SELECT count(*) FROM read_parquet('{parquet_path}')"
    ).to_df()

check.head()

,count_star()
0,100000


All 100,000 rows were downloaded successfully.

In [21]:
# check how many rows of each month are included
parquet_path = Path('data/blob_data_100k.parquet')

check = duckdb.sql(
        f"""
        SELECT "MONTH", count(*) as ct 
        FROM read_parquet('{parquet_path}') 
        where "YEAR" = '2024'
        group by "MONTH"
        """).to_df()

check.head()

,MONTH,ct
0,06,96802
1,07,3198


Note that not all months in the window are included in the first 100,000 rows, only June and July.

In [ ]:
# inspect data
parquet_path = Path('data/blob_data_100k.parquet')

df = duckdb.sql(f"""select * from read_parquet('{parquet_path}') limit 5;""").to_df()
df

,BLOB_ID,FULL_IMG_ID,POINT,SIZE,POLYGON_BOUNDRY_BOX,BLOB_POLYGON,GEO_HASHES,CONSTRUCTION_STAGE,BUILDING_TYPE,CS_MODEL_ID,...,IS_DUPLICATE,IS_BLOB_ON_IMAGE_EDGE,IS_VALID,IS_IMPUTED,IS_EXCLUDED,COUNTY,IS_OVERLAPPING_FOOTPRINT,IS_OVERLAPPING_ROAD,FOOTPRINT_ID,ROAD_DATA_ID
0,BID_a7258c95-7e44-424d-a227-1348f791ba39,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.4602132804818 29.46763817191083),452,POLYGON ((-98.4602662037037 29.467706018518502...,POLYGON ((-98.4602662037037 29.467645833333318...,9v1zxkr,ROOF,NON-SF,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,True,False,FP_ID_1a055cc6-21f7-416a-90a3-f8afd3e13b1c,None
1,BID_b433a935-c506-4937-9be8-9b63b87ca696,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.42449395672409 29.509311504753402),935,POLYGON ((-98.42457175925924 29.50939120370369...,POLYGON ((-98.42457175925924 29.50937268518517...,9v4pb7c|9v4pbk1,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,True,False,FP_ID_0e97e0c6-451c-4a83-a304-22d6a698cb19,None
2,BID_d9fa48dd-5ee7-4258-9d56-264d38f2eecc,FIID_87c905a3-ddd6-4a93-84cb-f07be70e1327_2024...,POINT (-96.87599626765075 32.86770283696455),4396,POLYGON ((-96.87620601851854 32.86782175925923...,POLYGON ((-96.87620601851854 32.86776157407404...,9vg4gx3|9vg4gx2,ROOF,NON-SF,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,True,False,True,False,False,DALLAS,True,False,FP_ID_41e0c77e-db4b-4702-bae9-17f823ef2cd8,None
3,BID_44517c93-b035-42b5-90dd-de2e341aa1d8,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.42545971924973 29.455358060608745),623,POLYGON ((-98.4255162037037 29.455418981481465...,POLYGON ((-98.4255162037037 29.455418981481465...,9v4p860,ROOF,BACKGROUND,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,False,False,None,None
4,BID_cd928abd-adbf-4481-bc2f-627c7e0e27f9,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.43521898353451 29.502309820768644),451,POLYGON ((-98.43526620370369 29.50235879629628...,POLYGON ((-98.43526620370369 29.50234953703702...,9v4pb49,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,False,False,None,None


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   BLOB_ID                   5 non-null      object
 1   FULL_IMG_ID               5 non-null      object
 2   POINT                     5 non-null      object
 3   SIZE                      5 non-null      int64 
 4   POLYGON_BOUNDRY_BOX       5 non-null      object
 5   BLOB_POLYGON              5 non-null      object
 6   GEO_HASHES                5 non-null      object
 7   CONSTRUCTION_STAGE        5 non-null      object
 8   BUILDING_TYPE             5 non-null      object
 9   CS_MODEL_ID               5 non-null      object
 10  BT_MODEL_ID               5 non-null      object
 11  CITY                      5 non-null      object
 12  STATE                     5 non-null      object
 13  YEAR                      5 non-null      object
 14  MONTH                     5 no

Get schema of the downloaded dataset:

In [ ]:
parquet_path = Path('data/blob_data_100k.parquet')
duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path}')").show()

┌──────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name        │ column_type │  null   │   key   │ default │  extra  │
│         varchar          │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ BLOB_ID                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ FULL_IMG_ID              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ POINT                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ SIZE                     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ POLYGON_BOUNDRY_BOX      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BLOB_POLYGON             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ GEO_HASHES               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CONSTRUCTION_STAGE       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BU

In [23]:
print(f"This slice of data is about {100000/21946235 * 100:.3f}% of the total size of the dataset.")

This slice of data is about 0.456% of the total size of the dataset.


Create synthetic dataset to test our retrieve data function:

In [ ]:
# Reload helpers if needed.
importlib.reload(h)

# Create a fake dataset with 10,000 rows.
num_rows = 1000
months = ['03', '04', '05', '06', '07']
data = {
    "BLOB_ID": [f"blob_{i}" for i in range(num_rows)],
    "YEAR": ['2024'] * num_rows,
    # Cycle through the months list.
    "MONTH": [months[i % len(months)] for i in range(num_rows)],
    "VALUE": [i for i in range(num_rows)],
    # Simulate CTID as an integer that increases sequentially.
    "ctid": [i for i in range(num_rows)]
}
fake_df = pd.DataFrame(data)

# Create a file-based SQLite database.
sqlite_db_path = "fake_blob.db"
engine = create_engine(f"sqlite:///{sqlite_db_path}")
# Save the fake DataFrame to a table named 'blob'.
fake_df.to_sql("blob", engine, if_exists="replace", index=False)

# Ensure the data directory exists.
os.makedirs("data", exist_ok=True)

# Remove any existing checkpoint file to start fresh.
checkpoint_path = Path("data/chunk_checkpoint.json")
if checkpoint_path.exists():
    checkpoint_path.unlink()

# Now run our retrieve_data function using the SQLite engine.
# In this test, we set chunk_size=2500 to produce 4 chunks.
h.retrieve_data(chunk_size=500, engine_url=f"sqlite:///{sqlite_db_path}")

Function `retrieve_data` executed in 0.0075 sec, CPU: 4.10%, Memory: 11.19MB


# Run function to download data

Now we will run the function on the actual dataset. The way the function is written, it can pick up where it left off by relying on checkpoint files.

In [ ]:
# Reload helpers if needed
importlib.reload(h)

h.retrieve_data(chunk_size=100000, max_chunks=10, prefix='blob_20250307')

Starting chunk 7. Last CTID: (11010946,3)
Function `retrieve_data` executed in 80.5807 sec, CPU: 5.30%, Memory: 188.38MB


We encountered an issue where there was no more room in the tmp folder on the database side, but we managed to download 6 chunks of data, or 600,000 rows.

Let's now inspect our downloaded chunks:

In [25]:
# test count(*) of created dataset

parquet_path = Path('archive/blob_20250307_chunk_1.parquet')

check = duckdb.sql(
        f"SELECT count(*) FROM read_parquet('{parquet_path}')"
    ).to_df()

check

,count_star()
0,100000


In [26]:
parquet_path = Path('archive/blob_20250307_chunk_1.parquet')

check = duckdb.sql(
        f"""
        SELECT "MONTH" ,count(*) 
        FROM read_parquet('{parquet_path}')
        group by "MONTH"
        order by "MONTH"
        """
    ).to_df()

check

,MONTH,count_star()
0,03,322
1,04,1
2,05,13
3,06,95895
4,07,3769


In [28]:
parquet_path = Path('archive/blob_20250307_chunk_1.parquet')

check = duckdb.sql(
        f"""
        SELECT * 
        FROM read_parquet('{parquet_path}')
        """
    ).to_df()

check.head(3)

,BLOB_ID,FULL_IMG_ID,POINT,SIZE,POLYGON_BOUNDRY_BOX,BLOB_POLYGON,GEO_HASHES,CONSTRUCTION_STAGE,BUILDING_TYPE,CS_MODEL_ID,...,IS_BLOB_ON_IMAGE_EDGE,IS_VALID,IS_IMPUTED,IS_EXCLUDED,COUNTY,IS_OVERLAPPING_FOOTPRINT,IS_OVERLAPPING_ROAD,FOOTPRINT_ID,ROAD_DATA_ID,pgt_ctid
0,BID_4b0de362-432b-4169-880b-2502fe191e45,FIID_eeca1e04-8ba1-43cb-931c-81c560491477_2024...,POINT (-96.57220292317847 32.777977159601775),519,POLYGON ((-96.57224768518518 32.77805324074072...,POLYGON ((-96.57224768518518 32.77797916666665...,9vg66rn,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,DALLAS,True,False,FP_ID_36b25e18-56d6-482d-8f54-dac7c70c6687,None,"(1000,6)"
1,BID_8ae754b5-026d-4aac-9b9c-7b473bcf1f79,FIID_46cf3f0f-2345-41e0-8e50-fcf561da16c4_2024...,POINT (-95.33409120370364 29.674920974576246),1180,POLYGON ((-95.33415972222215 29.67505787037034...,POLYGON ((-95.33415972222215 29.67503935185182...,9vk0yd5,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,HARRIS,True,False,FP_ID_c7fa7535-5ebb-4132-9814-e695c737a316,None,"(1000000,6)"
2,BID_ea519744-3092-4bb3-a340-19afe400056f,FIID_f7968076-35bc-42c8-b417-f88e8fafd619_2024...,POINT (-96.74267940156228 33.196998841454544),1017,POLYGON ((-96.74277546296292 33.19707175925927...,POLYGON ((-96.74277546296292 33.19699305555556...,9vghye6,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,COLLIN,True,False,FP_ID_6f9bcbaf-4075-4ceb-aab1-ccdc562744e7,None,"(10000001,2)"


# Concatenate Downloaded Data

In [29]:
# define data location
data_location = Path('archive')

# retrieve all relevant parquet files
data_files = list(Path(data_location).glob('blob_20250307_chunk_*.parquet'))

data_files

[PosixPath('archive/blob_20250307_chunk_3.parquet'),
 PosixPath('archive/blob_20250307_chunk_2.parquet'),
 PosixPath('archive/blob_20250307_chunk_1.parquet'),
 PosixPath('archive/blob_20250307_chunk_4.parquet'),
 PosixPath('archive/blob_20250307_chunk_5.parquet'),
 PosixPath('archive/blob_20250307_chunk_6.parquet')]

Concatenate data:

In [12]:
df = duckdb.sql("select * from 'data/blob_20250307_chunk_*.parquet'").to_df()
df.head()

,BLOB_ID,FULL_IMG_ID,POINT,SIZE,POLYGON_BOUNDRY_BOX,BLOB_POLYGON,GEO_HASHES,CONSTRUCTION_STAGE,BUILDING_TYPE,CS_MODEL_ID,...,IS_BLOB_ON_IMAGE_EDGE,IS_VALID,IS_IMPUTED,IS_EXCLUDED,COUNTY,IS_OVERLAPPING_FOOTPRINT,IS_OVERLAPPING_ROAD,FOOTPRINT_ID,ROAD_DATA_ID,pgt_ctid
0,BID_4b0de362-432b-4169-880b-2502fe191e45,FIID_eeca1e04-8ba1-43cb-931c-81c560491477_2024...,POINT (-96.57220292317847 32.777977159601775),519,POLYGON ((-96.57224768518518 32.77805324074072...,POLYGON ((-96.57224768518518 32.77797916666665...,9vg66rn,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,DALLAS,True,False,FP_ID_36b25e18-56d6-482d-8f54-dac7c70c6687,None,"(1000,6)"
1,BID_8ae754b5-026d-4aac-9b9c-7b473bcf1f79,FIID_46cf3f0f-2345-41e0-8e50-fcf561da16c4_2024...,POINT (-95.33409120370364 29.674920974576246),1180,POLYGON ((-95.33415972222215 29.67505787037034...,POLYGON ((-95.33415972222215 29.67503935185182...,9vk0yd5,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,HARRIS,True,False,FP_ID_c7fa7535-5ebb-4132-9814-e695c737a316,None,"(1000000,6)"
2,BID_ea519744-3092-4bb3-a340-19afe400056f,FIID_f7968076-35bc-42c8-b417-f88e8fafd619_2024...,POINT (-96.74267940156228 33.196998841454544),1017,POLYGON ((-96.74277546296292 33.19707175925927...,POLYGON ((-96.74277546296292 33.19699305555556...,9vghye6,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,COLLIN,True,False,FP_ID_6f9bcbaf-4075-4ceb-aab1-ccdc562744e7,None,"(10000001,2)"
3,BID_ea962ddc-c9ad-4a0a-905e-2012e0d6485d,FIID_a2ae6d14-612c-4123-bd66-636f6821c091_2024...,POINT (-96.72505400860906 33.16324660904856),826,POLYGON ((-96.72512731481478 33.16331712962967...,POLYGON ((-96.72512731481478 33.16324768518522...,9vghwvp|9vghwvn,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,COLLIN,True,False,FP_ID_3d617f10-f756-462f-b098-00badc42438a,None,"(10000002,6)"
4,BID_ea9a59f0-7f15-4371-ac86-096cf4eeaf3f,FIID_f7968076-35bc-42c8-b417-f88e8fafd619_2024...,POINT (-96.7388098211761 33.202477229997186),1310,POLYGON ((-96.73887731481477 33.20259027777779...,POLYGON ((-96.73887731481477 33.20255787037038...,9vghysm|9vghysk,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,True,False,False,COLLIN,True,False,FP_ID_3aa44e93-ea83-4f15-b570-078dad879910,None,"(10000003,2)"


Confirm shape of concatenated dataset:

In [30]:
df.shape

(600000, 27)

Save downloaded data to new parquet file:

In [38]:
concatenated_parquet = Path('data/blob_20250307.parquet')
df.to_parquet('data/blob_20250307.parquet')
print('Done.')

Done.


Save downloaded data to a geojson file to view in QGIS:

In [42]:
# save version of data to geojson to confirm in QGIS
blobs_geojson = Path('data/blob_20250307.geojson')

# convert the 'BLOB_POLYGON' column from WKT to geometry objects
df['geometry'] = df[
    'BLOB_POLYGON'].apply(lambda x: wkt.loads(x) if isinstance(x, str) else x)

# create a GeoDataFrame, specifying the geometry column and CRS
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# write GeoDataFrame to GeoJSON file
if not blobs_geojson.is_file():
    gdf.to_file('data/blob_20250307.geojson', driver='GeoJSON')

print('GeoJSON file saved successfully.')

GeoJSON file saved successfully.
